# # 🚀 Sistema de copy trading

In [1]:
import asyncio
import os
from dotenv import load_dotenv

from copy_trading import CopyTrading, CopyTradingConfig

load_dotenv()

🎯 Módulo PumpFun cargado
🚀 Funcionalidades disponibles:
   - Fetcher de precios directo de bonding curve
   - Análisis de estado de curve
   - Cálculo de progreso de bonding
   - Precios en tiempo real desde Pump.fun
   - Cliente API centralizado con WebSocket y HTTP
   - Trading con APIs Lightning y Local
   - Gestión de wallets Lightning
   - Creación de tokens
   - Soporte para API key en WebSocket (PumpSwap data)
   - Parser asíncrono de transacciones de Pump.fun
   - Análisis de actividad de traders
   - Patrón Singleton y Context Manager
🎯 Módulo Copy Trading Mini cargado
🚀 Sistema simplificado de copy trading
   ✅ Integración directa con PumpFun
   ✅ Replicación automática de trades
   ✅ Gestión de posiciones FIFO
   ✅ Validaciones configurables
   ✅ Logging estructurado
   ✅ Soporte Lightning y Local Trade


True

In [2]:
async def main():
    """Función principal para ejecutar el sistema con notificaciones."""
    
    # 1. Configurar las credenciales de Telegram
    # Puedes obtener estas credenciales de:
    # - Bot Token: Crear un bot con @BotFather en Telegram
    # - Chat ID: Enviar mensaje a tu bot y usar la API de Telegram para obtener el chat_id

    telegram_bot_token = os.getenv("TELEGRAM_BOT_TOKEN")  # Token del bot
    telegram_chat_id = os.getenv("TELEGRAM_CHAT_ID")     # ID del chat

    if not telegram_bot_token or not telegram_chat_id:
        print("⚠️ ADVERTENCIA: Credenciales de Telegram no configuradas")
        print("   Configura las variables de entorno:")
        print("   - TELEGRAM_BOT_TOKEN: Token de tu bot de Telegram")
        print("   - TELEGRAM_CHAT_ID: ID del chat donde recibir notificaciones")
        print("   El sistema funcionará sin notificaciones de Telegram")

    # 2. Configurar traders a seguir
    traders_to_copy = [
        "3q1WCMp25FQcs1JV86H5Y7XPfxjudiMudtsCBGfgCwj7", # Ejemplo de trader 1
        "8HEFiQSKqmEJ65UVEmHEyjvieZhrHRdLM7N4KuKpLS3Y", # Ejemplo de trader 2
        "A1Uh8udZg2CMHS5kKeQouYRGfiAxJ7MYMuRbAz913hJX", # Ejemplo de trader 3
        "GMN2f6PsBwUKKpuxQdsJGveJKsNEgiH7APVwdfBtnAtz", # Ejemplo de trader 4
    ]

    # 3. Crear configuración con notificaciones habilitadas
    config = CopyTradingConfig(
        traders=traders_to_copy,
        wallet_file="wallets/wallet_pumpportal.json",

        # Configuración de copy trading
        amount_mode="exact",
        amount_value=10,  # 10% de cada trade

        # Configuración de notificaciones
        notifications_enabled=False,
        telegram_bot_token=telegram_bot_token,
        telegram_chat_id=telegram_chat_id,
        telegram_messages_per_minute=30,  # Límite de mensajes por minuto

        # Configuración de trading
        transaction_type="lightning_trade",
        pool_type="auto",
        slippage_tolerance=0.15,  # 15%
        priority_fee_sol=0.00005,

        # Configuración de seguridad
        max_position_size=0.01,  # Máximo 0.01 SOL por posición
        max_daily_volume=0.1,    # Máximo 0.1 SOL diario

        # Modo de prueba
        dry_run=False  # Cambiar a False para trading real
    )

    # 4. Crear e iniciar el sistema
    system = CopyTrading(config=config)

    try:
        async with system:
            print(f"\n✅ Sistema iniciado con notificaciones")
            print(f"📱 Telegram: {'Habilitado' if telegram_bot_token and telegram_chat_id else 'Deshabilitado'}")
            print(f"🔄 Modo: {'DRY RUN' if config.dry_run else 'LIVE'}")
            print(f"👥 Siguiendo {len(config.traders)} traders")
            print("\nPresiona Ctrl+C para detener.")

            # Enviar notificación de prueba
            if system.notification_manager:
                status_msg = (
                    f"Sistema iniciado correctamente\n"
                    f"- Telegram: {'✅ Habilitado' if telegram_bot_token and telegram_chat_id else '❌ Deshabilitado'}\n"
                    f"- Modo: {'🔄 DRY RUN' if config.dry_run else '🚀 LIVE'}\n"
                    f"- Traders: 👥 {len(config.traders)}"
                )
                await system.notification_manager.notify_system(status_msg, "success")

            # Bucle principal - mostrar métricas cada 5 minutos
            while system.is_running:
                await asyncio.sleep(300)  # 5 minutos

                try:
                    metrics = await system.get_metrics()

                    print("\n--- Métricas del Sistema ---")
                    print(f"  - Tiempo activo: {metrics['system_metrics']['uptime_seconds']:.0f}s")
                    print(f"  - Trades recibidos: {metrics['callback_stats']['trades_received']}")
                    print(f"  - Trades validados: {metrics['callback_stats']['trades_validated']}")
                    print(f"  - Trades en cola: {metrics['queue_stats']['pending_count']}")
                    print(f"  - Trades ejecutados: {metrics['system_metrics']['trades_executed']}")
                    print(f"  - Volumen total: {metrics['system_metrics']['total_volume_sol']:.6f} SOL")
                    print(f"  - Balance actual: {metrics['wallet_balance']:.6f} SOL")
                    print("--------------------------\n")

                    # Enviar resumen por Telegram cada hora
                    if system.notification_manager and metrics['system_metrics']['uptime_seconds'] % 3600 < 300:
                        metrics_msg = (
                            f"Resumen horario del sistema\n"
                            f"- Tiempo activo: ⏱️ {metrics['system_metrics']['uptime_seconds']/3600:.1f}h\n"
                            f"- Trades recibidos: 📥 {metrics['callback_stats']['trades_received']}\n"
                            f"- Trades ejecutados: ✅ {metrics['system_metrics']['trades_executed']}\n"
                            f"- Volumen: 💰 {metrics['system_metrics']['total_volume_sol']:.6f} SOL\n"
                            f"- Balance: 💎 {metrics['wallet_balance']:.6f} SOL"
                        )
                        await system.notification_manager.notify_system(metrics_msg, "info")

                except Exception as e:
                    print(f"Error obteniendo métricas: {e}")

    except KeyboardInterrupt:
        print("\nInterrupción del usuario detectada.")
    except Exception as e:
        print(f"Error en el sistema: {e}")

        # Notificar error crítico
        if system.notification_manager:
            error_msg = (
                f"Error crítico en el sistema:\n"
                f"- Tipo: ⚠️ {type(e).__name__}\n"
                f"- Mensaje: 🔥 {str(e)}"
            )
            await system.notification_manager.notify_system(error_msg, "error")

    print("🏁 Sistema detenido.")

In [3]:
try:
    await main()
except asyncio.CancelledError:
    print("Programa cancelado por el usuario")

2025-07-10 22:52:50 - CopyTradingMini - INFO - 🚀 Iniciando sistema Copy Trading Mini
2025-07-10 22:52:50 - CopyTradingMini - INFO - Modo: LIVE
2025-07-10 22:52:50 - CopyTradingMini - INFO - Traders a seguir: 4
✅ Wallet validada correctamente: 62bzYQAb...
🌐 PumpFun API Client inicializado
   WebSocket: ✅ wss://pumpportal.fun/api/data
   HTTP: ✅ https://pumpportal.fun/api
   API Key: ✅ 9wvpykaa...
🔗 Conectando WebSocket... (intento 1)
✅ Sesión HTTP inicializada
✅ WebSocket conectado exitosamente
2025-07-10 22:52:51 - CopyTradingMini - INFO - ✅ Cliente API conectado
2025-07-10 22:52:51 - CopyTradingMini - INFO - Wallet cargada: 62bzYQAbEhtc7eP3qb9G557fhLqcfNxeTKr6v1UshnD1
2025-07-10 22:52:51 - CopyTradingMini - INFO - API Key configurada: 9wvpykaa9x57jtu29th7...
🌐 Conectado a Solana mainnet-beta (RPC: https://api.mainnet-beta.solana.com/)
2025-07-10 22:52:53 - CopyTradingMini - INFO - Validation engine inicializado
2025-07-10 22:52:53 - CopyTradingMini - INFO - Balance inicial: 0.027406 S